<a href="https://colab.research.google.com/github/ac1094/Vid-Processing/blob/main/Inconsistency_Detector1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [286]:
import import_ipynb
from Functions import *

In [287]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# Define the Optimized Transformer Model with Convolutional Layers
class OptimizedTransformer(nn.Module):
    def __init__(self):
        super(OptimizedTransformer, self).__init__()
        self.conv1 = nn.Conv2d(9, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 12 * 25, 512)  # Adjusted size
        self.fc2 = nn.Linear(512, 50 * 100)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        #print("conv1_in: ", x.size())
        x = self.conv1(x)  # (N, 32, 50, 100)
        #print("pool_in: ", x.size())
        x = self.pool(x)  # (N, 32, 25, 50)
        #print("conv2_in: ", x.size())
        x = self.conv2(x) # (N, 64, 25, 50)
        #print("pool1_in: ", x.size())
        x = self.pool1(x) # (N, 64, 12, 25)
        #print("fc1_in: ", x.size())
        x = x.view(-1, 64 * 12 * 25)  # Flatten
        x = self.fc1(x)  # (N, 512)
        #print("fc2_in: ", x.size())
        x = self.fc2(x)  # (N, 50 * 100)
        #print("sig_in: ", x.size())
        x = self.sigmoid(x)  # (N, 50 * 100)
        #print("view_in: ", x.size())
        x = x.view(2, 50, 100)  # Reshape to the desired output size
        return x

# Custom Dataset to handle lists of tensors
class TensorListDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = [torch.tensor(i) if isinstance(i, np.ndarray) else i for i in inputs]
        self.targets = [torch.tensor(t) if isinstance(t, np.ndarray) else t for t in targets]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

# Training function
def train_model(model, train_loader, num_epochs=10, learning_rate=0.001):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            target = target.squeeze(1)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader)}')

# Prediction function
def predict(model, input_tensor):
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        binary_output = (output > 0.5).float()
    return binary_output


In [288]:
F = read_images("saved_frames")
F = [cv2.resize(f,(50,100)) for f in F]
F[3] = F[3][:,:,:3]
F[6] = F[6][:,:,:3]
F[9] = F[9][:,:,:3]
F[13] = F[13][:,:,:3]
F = F[:15]
D = np.asarray([abs(F[i]-F[i+1]) for i in range(len(F)-1)],dtype=np.uint8)
D0 = np.asarray(D[:-1],dtype=np.uint8)
D2 = np.asarray(D[1:],dtype=np.uint8)
down = 0
up = 255
B = np.asarray([abs(cv2.Canny(cv2.cvtColor(F[i],cv2.COLOR_BGR2GRAY),down,up)-cv2.Canny(cv2.cvtColor(F[i+1],cv2.COLOR_BGR2GRAY),down,up)) for i in range(len(F)-1)],dtype=np.uint8)
B0 = np.asarray(B[:-1],dtype=np.uint8)
B2 = np.asarray(B[1:],dtype=np.uint8)
F0 = np.asarray(F[:-2],dtype=np.uint8)
F2 = np.asarray(F[2:],dtype=np.uint8)
F = np.asarray(F[1:-1],dtype=np.uint8)

In [289]:
Input = [np.asarray((F0[i][:,:,0],F0[i][:,:,1],F0[i][:,:,2],
                    F[i][:,:,0],F[i][:,:,1],F[i][:,:,2],
                    F2[i][:,:,0],F2[i][:,:,1],F2[i][:,:,2],
                    #D0[i][:,:,0],D0[i][:,:,1],D0[i][:,:,2],
                    #D2[i][:,:,0],D2[i][:,:,1],D2[i][:,:,2],
                    #B0[i],
                    #B2[i]
                     )) for i in range(len(F))]

In [290]:
print(Input[0].shape)
print(len(Input))

(9, 100, 50)
13


In [291]:
input_arrays = [torch.tensor(i,dtype=torch.float32) for i in Input[:-1]]
print(len(input_arrays),input_arrays[0].shape)

12 torch.Size([9, 100, 50])


In [292]:
target_arrays = [torch.zeros(1,50,100,dtype=torch.float32) for _ in range(len(Input[:-1]))]
print(len(target_arrays),target_arrays[0].shape)

12 torch.Size([1, 50, 100])


In [293]:
Masks = read_images("Masks")
Masks = [cv2.cvtColor(m[:,:,:3],cv2.COLOR_RGB2GRAY) for m in Masks]
Masks = [cv2.resize(m,(50,100)) for m in Masks] # Resize before normalization
Masks = [torch.tensor(m / 255.0, dtype=torch.float32) for m in Masks] # Normalize pixel values to [0, 1]
target_arrays[3] = Masks[0].reshape((1,50,100))
target_arrays[6] = Masks[1].reshape((1,50,100))
target_arrays[9] = Masks[2].reshape((1,50,100))

In [294]:
dataset = TensorListDataset(input_arrays, target_arrays)

In [295]:
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [296]:
# Instantiate the model
model = OptimizedTransformer()

In [297]:
# Train the model on the random data
train_model(model, train_loader, num_epochs=5)

Epoch 1/5, Loss: 26.287648518880207
Epoch 2/5, Loss: 8.166882197062174
Epoch 3/5, Loss: 5.113840977350871
Epoch 4/5, Loss: 4.118620991706848
Epoch 5/5, Loss: 4.126690308252971


In [302]:
# Create a new random input tensor for prediction
new_input = torch.randn(2, 9, 100, 50)

# Predict using the trained model
predicted_output = predict(model, new_input)

# Print the shape of the predicted output
print(predicted_output.size())  # Should output torch.Size([1, 320, 640])

# Optionally, print the predicted binary output (0s and 1s)
print(predicted_output)

torch.Size([2, 50, 100])
tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 0., 1.],
         [1., 1., 1.,  ..., 1., 1., 0.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 0.],
         [0., 0., 1.,  ..., 0., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 0., 1.],
         [1., 1., 1.,  ..., 1., 1., 0.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 0.],
         [0., 0., 1.,  ..., 0., 1., 1.]]])


In [ ]:
display_frame(predicted_output.reshape(320,640).astype(np.uint8))

In [ ]:
# Create a custom dataset and DataLoader
dataset = TensorListDataset(inputs, targets)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Instantiate the model
model = SimpleTransformer()

# Train the model on the random data
train_model(model, train_loader, num_epochs=5)

# Create a new random input tensor for prediction
new_input = torch.randn(1, 17, 320, 640)

# Predict using the trained model
predicted_output = predict(model, new_input)

# Print the shape of the predicted output
print(predicted_output.size())  # Should output torch.Size([1, 320, 640])

# Optionally, print the predicted binary output (0s and 1s)
print(predicted_output)